# Clean Data

In [3]:
# Import packages and datasets
import pandas as pd
import geopandas as gpd
import numpy as np
linked = pd.read_csv('../../fm-lm-demand-simulation/data/external/LINKED_Public.csv', low_memory=False)
household = pd.read_csv('../../fm-lm-demand-simulation/data/external/HH_Public.csv', low_memory=False)
person = pd.read_csv('../../fm-lm-demand-simulation/data/external/PER_Public.csv')

### Feature selected for analysis:

LINKED_Public:
 
- SAMPN (household identifier)
- PERNO (person number)
- PERTYPE (person type code)
- ORIG_HOME (if home is origin)
- DEST_HOME (if home is destination)
- DOW (day of week of travel)
- OTAZ (origin TAZ - NYMTC; WGS84) -- drop 0 Location Outside of NYBPM area
- DTAZ (destination TAZ - NYMTC; WGS84) -- drop 0 Location Outside of NYBPM area
- OTPURP_AGG (origin primary trip purpose)
- DTPURP_AGG (destination primary trip purpose)
- LTMODE_AGG (aggregate linked trip mode)
- TRP_DEP_HR (trip departure hours)
- TRP_DEP_MIN (trip departure minutes)
- TRP_ARR_HR (trip arrival hours)
- TRP_ARR_MIN (trip arrival minutes)
- TRPDUR (trip duration minutes)
- ACTDUR (activity duration)
- OTRACT (origin tract 2010; WGS84)
- DTRACT (destination tract 2010; WGS84)
- PMODE1 (primary mode - re-categorized based on PMODE)
- PMODE_R (primary mode range)
- PMODE_R2 (primary mode range2)
- PMODE_R3 (primary mode range3)
- WORK_PURP (work vs. non-work)
- DTPURP_R (aggregate destination primary trip purpose)
- ODTPURP (aggregate Origin to Destination primary trip purpose)
- ODTPURP1
- ODTPURP2
- ODTPURP2_R
- PLOC
- WORKTRIP (dummy for work trip based on ODTPURP2_R)
- TOD_R (time of departure range)
- TOD_PEAK (time of departure: non-peak vs. peak)
- TRIPDIST_R1 (trip distance range 1)
- TRIPDIST_R2 (trip distance range 2)


HH_Public:

- STYPE (sample type) -- Matched
- CTFIP (household residential location) -- NYC five boroughs(36047; 36061; 36081; 36005; 36085)
- INCOM (household income) -- drop 99 Refused
- DOW (day of travel during the week)
- HHVEH (number of household vehicles)


PER_Public:

- PSAMP (unique Person Identifier)
- SAMPN (household identifier)
- PERNO (person identifier)
- Gender -- drop 9 Refused
- AGE_R (age range) -- drop 9 Age not provided
- LIC (valid driver’s license) -- drop 8 Don’t Know and 9 Refused
- RACE (Race/Ethnicity) -- drop 98 Don’t Know, and 99 Refused
- EMPLY (emplyed) -- drop 8 Don’t Know and 9 Refused
- WORKS (work status)
- WKSTAT (unemployment Status) -- drop 8 Don’t Know and 9 Refused
- JOBS (number of jobs)
- WDAYS (work days) -- drop 8 Don’t Know and 9 Refused
- INDUS (industry)
- OCCUP (occupation)
- O_OCCUP (other in occupation)
- WLOC (work location)
- WFIPS (work location county) -- NYC five boroughs(36047; 36061; 36081; 36005; 36085)
- WMODE (usual mode of transportation to work) -- drop 98 Don’t Know and 99 Refused
- O_WMODE (Other, model of transportation to work)
- TTTWS (typical travel time to work) -- drop 999 Refused
- CDRIV (as part of job, does/dp drive a household vehicle from work site to work site?) -- drop 8 Don’t Know and 9 Refused
- WSTRT (work start time) -- drop 9999 Don’t Know / Refused
- WEND (work end time) -- drop 9999 Don’t Know / Refused
- STRVR (start time variation) -- drop 7 Other, 8 Don’t Know and 9 Refused
- ENDVR (end time variation) -- drop 7 Other, 8 Don’t Know and 9 Refused
- STUDE (student status) -- drop 8 DK and 9 RF



In [4]:
# Select needed features from datasets
linked_clean = linked[['SAMPN', 'PERNO', 'PERTYPE', 'ORIG_HOME', 'DEST_HOME', 'DOW', 'OTAZ', 'DTAZ', 'DTPURP', 'OTPURP_AGG', 'DTPURP_AGG', 'LTMODE_AGG',
                       'TRP_DEP_HR', 'TRP_DEP_MIN', 'TRP_ARR_HR', 'TRP_ARR_MIN', 'TRPDUR_R', 'ACTDUR', 'OTRACT', 'DTRACT', 'PMODE1', 'PMODE_R',
                       'PMODE_R2', 'PMODE_R3', 'WORK_PURP', 'DTPURP_R', 'ODTPURP', 'ODTPURP1', 'ODTPURP2', 'ODTPURP2_R', 'PLOC', 'WORKTRIP', 'TOD_R',
                       'TOD_PEAK', 'TRIPDIST_R1', 'TRIPDIST_R2']]

household_clean = household[['SAMPN', 'STYPE', 'CTFIP', 'INCOM_R', 'DOW', 'HHVEH']]

person_clean = person[['SAMPN', 'PERNO', 'GENDER', 'AGE_R', 'LIC', 'RELAT', 'RACE', 'EMPLY', 'WORKS', 'WKSTAT', 'JOBS', 'WDAYS', 'INDUS', 'OCCUP',
                       'O_OCCUP', 'WLOC', 'WFIPS', 'WMODE', 'O_WMODE', 'TTTWS', 'CDRIV', 'WSTRT', 'WEND', 'STRVR', 'ENDVR', 'STUDE']]

# Clean Linked Trips Dataset

In [5]:
len(linked_clean)

143925

In [6]:
linked_clean.isnull().any()

SAMPN          False
PERNO          False
PERTYPE        False
ORIG_HOME      False
DEST_HOME      False
DOW            False
OTAZ           False
DTAZ           False
DTPURP         False
OTPURP_AGG     False
DTPURP_AGG     False
LTMODE_AGG     False
TRP_DEP_HR     False
TRP_DEP_MIN    False
TRP_ARR_HR     False
TRP_ARR_MIN    False
TRPDUR_R       False
ACTDUR         False
OTRACT         False
DTRACT         False
PMODE1         False
PMODE_R        False
PMODE_R2       False
PMODE_R3       False
WORK_PURP      False
DTPURP_R       False
ODTPURP        False
ODTPURP1       False
ODTPURP2       False
ODTPURP2_R     False
PLOC            True
WORKTRIP       False
TOD_R          False
TOD_PEAK        True
TRIPDIST_R1     True
TRIPDIST_R2     True
dtype: bool

In [7]:
linked_clean = linked_clean[np.isfinite(linked_clean['TOD_PEAK'])]
linked_clean = linked_clean[np.isfinite(linked_clean['TRIPDIST_R1'])]
linked_clean = linked_clean[np.isfinite(linked_clean['TRIPDIST_R2'])]


In [8]:
len(linked_clean)

140670

In [9]:
linked_clean.isnull().any()

SAMPN          False
PERNO          False
PERTYPE        False
ORIG_HOME      False
DEST_HOME      False
DOW            False
OTAZ           False
DTAZ           False
DTPURP         False
OTPURP_AGG     False
DTPURP_AGG     False
LTMODE_AGG     False
TRP_DEP_HR     False
TRP_DEP_MIN    False
TRP_ARR_HR     False
TRP_ARR_MIN    False
TRPDUR_R       False
ACTDUR         False
OTRACT         False
DTRACT         False
PMODE1         False
PMODE_R        False
PMODE_R2       False
PMODE_R3       False
WORK_PURP      False
DTPURP_R       False
ODTPURP        False
ODTPURP1       False
ODTPURP2       False
ODTPURP2_R     False
PLOC            True
WORKTRIP       False
TOD_R          False
TOD_PEAK       False
TRIPDIST_R1    False
TRIPDIST_R2    False
dtype: bool

In [10]:
linked_clean = linked_clean.loc[linked_clean['OTAZ'] != 0]

In [11]:
linked_clean = linked_clean.loc[linked_clean['DTAZ'] != 0]

In [12]:
linked_clean = linked_clean.loc[linked_clean['PMODE1'] != 97]
linked_clean = linked_clean.loc[linked_clean['PMODE_R'] != 97]
linked_clean = linked_clean.loc[linked_clean['PMODE_R2'] != 97]
linked_clean = linked_clean.loc[linked_clean['PMODE_R3'] != 97]

In [13]:
len(linked_clean)

119333

In [14]:
linked_clean.head()

,SAMPN,PERNO,PERTYPE,ORIG_HOME,DEST_HOME,DOW,OTAZ,DTAZ,DTPURP,OTPURP_AGG,...,ODTPURP,ODTPURP1,ODTPURP2,ODTPURP2_R,PLOC,WORKTRIP,TOD_R,TOD_PEAK,TRIPDIST_R1,TRIPDIST_R2
0,3000056,1,2,1,0,3,2626,2627,24,0,...,3,3,4,3,1.0,2,1,1.0,2.0,3.0
1,3000056,1,2,0,1,3,2627,2626,3,9,...,6,10,4,3,NaN,2,1,1.0,2.0,3.0
2,3000056,1,2,1,0,3,2626,2627,9,0,...,1,1,1,1,1.0,1,2,0.0,2.0,3.0
3,3000056,1,2,0,1,3,2627,2626,3,1,...,4,8,1,1,NaN,1,2,0.0,2.0,3.0
4,3000056,1,2,1,0,3,2626,2628,25,0,...,3,3,4,3,1.0,2,3,1.0,2.0,3.0


In [15]:
linked_clean.to_csv('../../fm-lm-demand-simulation/data/processed/linked_trips.csv')

# Clean Household Dataset

In [16]:
len(household_clean)

18965

In [17]:
household_clean.isnull().any()

SAMPN      False
STYPE      False
CTFIP      False
INCOM_R    False
DOW        False
HHVEH      False
dtype: bool

In [18]:
household_clean = household_clean.loc[household_clean['INCOM_R'] != 99]


In [19]:
len(household_clean)

18965

In [20]:
household_clean.head()

,SAMPN,STYPE,CTFIP,INCOM_R,DOW,HHVEH
0,3000056,1,9001,3,3,3
1,3000128,1,36027,2,4,2
2,3000139,1,34027,9,2,2
3,3000146,1,34027,3,2,1
4,3000157,1,34017,1,3,0


In [21]:
household_clean.to_csv('../../fm-lm-demand-simulation/data/processed/household.csv')

# Clean Person Dataset

In [22]:
len(person_clean)

43558

In [23]:
person_clean.isnull().any()

SAMPN      False
PERNO      False
GENDER     False
AGE_R      False
LIC         True
RELAT      False
RACE       False
EMPLY       True
WORKS       True
WKSTAT      True
JOBS        True
WDAYS       True
INDUS       True
OCCUP       True
O_OCCUP    False
WLOC        True
WFIPS       True
WMODE       True
O_WMODE    False
TTTWS       True
CDRIV       True
WSTRT      False
WEND       False
STRVR       True
ENDVR       True
STUDE      False
dtype: bool

In [24]:
person_clean.loc[person_clean.AGE_R == 'Younger than 16 years', 'LIC'] = 'Not Applicable'
person_clean.loc[person_clean.AGE_R == 'Younger than 16 years', 'EMPLY'] = 'Not Applicable'
person_clean.loc[person_clean.AGE_R == 'Younger than 16 years', 'WORKS'] = 'Not Applicable'
person_clean.loc[person_clean.AGE_R == 'Younger than 16 years', 'WKSTAT'] = 'Not Applicable'
person_clean.loc[person_clean.AGE_R == 'Younger than 16 years', 'JOBS'] = 'Not Applicable'
person_clean.loc[person_clean.AGE_R == 'Younger than 16 years', 'WDAYS'] = 'Not Applicable'
person_clean.loc[person_clean.AGE_R == 'Younger than 16 years', 'INDUS'] = 'Not Applicable'
person_clean.loc[person_clean.AGE_R == 'Younger than 16 years', 'OCCUP'] = 'Not Applicable'
person_clean.loc[person_clean.AGE_R == 'Younger than 16 years', 'O_OCCUP'] = 'Not Applicable'
person_clean.loc[person_clean.AGE_R == 'Younger than 16 years', 'WLOC'] = 'Not Applicable'
person_clean.loc[person_clean.AGE_R == 'Younger than 16 years', 'WFIPS'] = 'Not Applicable'
person_clean.loc[person_clean.AGE_R == 'Younger than 16 years', 'WMODE'] = 'Not Applicable'
person_clean.loc[person_clean.AGE_R == 'Younger than 16 years', 'O_WMODE'] = 'Not Applicable'
person_clean.loc[person_clean.AGE_R == 'Younger than 16 years', 'TTTWS'] = 'Not Applicable'
person_clean.loc[person_clean.AGE_R == 'Younger than 16 years', 'CDRIV'] = 'Not Applicable'
person_clean.loc[person_clean.AGE_R == 'Younger than 16 years', 'WSTRT'] = 'Not Applicable'
person_clean.loc[person_clean.AGE_R == 'Younger than 16 years', 'WEND'] = 'Not Applicable'
person_clean.loc[person_clean.AGE_R == 'Younger than 16 years', 'ENDVR'] = 'Not Applicable'
person_clean.loc[person_clean.AGE_R == 'Younger than 16 years', 'STRVR'] = 'Not Applicable'


/Users/Danny1/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [25]:
person_clean.loc[person_clean.WORKS == 'Works', 'WKSTAT'] = 'Employed'
person_clean.loc[person_clean.WORKS == 'Not a Worker', 'JOBS'] = 'Not Applicable'
person_clean.loc[person_clean.WORKS == 'Not a Worker', 'WDAYS'] = 'Not Applicable'
person_clean.loc[person_clean.WORKS == 'Not a Worker', 'INDUS'] = 'Not Applicable'
person_clean.loc[person_clean.WORKS == 'Not a Worker', 'OCCUP'] = 'Not Applicable'
person_clean.loc[person_clean.WORKS == 'Not a Worker', 'O_OCCUP'] = 'Not Applicable'
person_clean.loc[person_clean.WORKS == 'Not a Worker', 'WLOC'] = 'Not Applicable'
person_clean.loc[person_clean.WORKS == 'Not a Worker', 'WFIPS'] = 'Not Applicable'
person_clean.loc[person_clean.WORKS == 'Not a Worker', 'WMODE'] = 'Not Applicable'
person_clean.loc[person_clean.WORKS == 'Not a Worker', 'TTTWS'] = 'Not Applicable'
person_clean.loc[person_clean.WORKS == 'Not a Worker', 'CDRIV'] = 'Not Applicable'
person_clean.loc[person_clean.WORKS == 'Not a Worker', 'ENDVR'] = 'Not Applicable'
person_clean.loc[person_clean.WORKS == 'Not a Worker', 'STRVR'] = 'Not Applicable'

In [26]:
person_clean.isnull().any()

SAMPN      False
PERNO      False
GENDER     False
AGE_R      False
LIC         True
RELAT      False
RACE       False
EMPLY       True
WORKS       True
WKSTAT      True
JOBS        True
WDAYS       True
INDUS       True
OCCUP       True
O_OCCUP    False
WLOC        True
WFIPS       True
WMODE       True
O_WMODE    False
TTTWS       True
CDRIV       True
WSTRT      False
WEND       False
STRVR       True
ENDVR       True
STUDE      False
dtype: bool

In [27]:
len(person_clean)

43558

In [28]:
person_clean = person_clean.loc[person_clean['GENDER'] != 'RF']
person_clean = person_clean.loc[person_clean['AGE_R'] != 'Age not provided']
person_clean = person_clean.loc[person_clean['LIC'] != 'RF']
person_clean = person_clean.loc[person_clean['LIC'] != 'DK']
person_clean = person_clean.loc[person_clean['RACE'] != 'RF']
person_clean = person_clean.loc[person_clean['RACE'] != 'DK']
person_clean = person_clean.loc[person_clean['EMPLY'] != 'RF']
person_clean = person_clean.loc[person_clean['EMPLY'] != 'DK']
person_clean = person_clean.loc[person_clean['WKSTAT'] != 'RF']
person_clean = person_clean.loc[person_clean['WKSTAT'] != 'DK']
person_clean = person_clean.loc[person_clean['WDAYS'] != 'RF']
person_clean = person_clean.loc[person_clean['WDAYS'] != 'DK']
person_clean = person_clean.loc[person_clean['WMODE'] != 'RF']
person_clean = person_clean.loc[person_clean['WMODE'] != 'DK']
person_clean = person_clean.loc[person_clean['TTTWS'] != 'RF']
person_clean = person_clean.loc[person_clean['CDRIV'] != 'RF']
person_clean = person_clean.loc[person_clean['CDRIV'] != 'DK']
person_clean = person_clean.loc[person_clean['WSTRT'] != 'RF']
person_clean = person_clean.loc[person_clean['WSTRT'] != 'DK']
person_clean = person_clean.loc[person_clean['WEND'] != 'RF']
person_clean = person_clean.loc[person_clean['WEND'] != 'DK']
person_clean = person_clean.loc[person_clean['STRVR'] != 'RF']
person_clean = person_clean.loc[person_clean['STRVR'] != 'DK']
person_clean = person_clean.loc[person_clean['ENDVR'] != 'RF']
person_clean = person_clean.loc[person_clean['ENDVR'] != 'DK']
person_clean = person_clean.loc[person_clean['STUDE'] != 'RF']
person_clean = person_clean.loc[person_clean['STUDE'] != 'DK']


In [29]:
len(person_clean)

40542

In [30]:
person_clean.isnull().sum()

SAMPN         0
PERNO         0
GENDER        0
AGE_R         0
LIC           0
RELAT         0
RACE          0
EMPLY         0
WORKS         0
WKSTAT        0
JOBS          0
WDAYS         0
INDUS         0
OCCUP         0
O_OCCUP       0
WLOC          0
WFIPS      1886
WMODE      1886
O_WMODE       0
TTTWS      1886
CDRIV      5663
WSTRT         0
WEND          0
STRVR      1886
ENDVR      1886
STUDE         0
dtype: int64

In [31]:
person_clean.head()

,SAMPN,PERNO,GENDER,AGE_R,LIC,RELAT,RACE,EMPLY,WORKS,WKSTAT,...,WFIPS,WMODE,O_WMODE,TTTWS,CDRIV,WSTRT,WEND,STRVR,ENDVR,STUDE
0,3000056,1,Female,35-54 years,Yes,Self,White,Yes,Works,Employed,...,Fairfield,Auto Passenger,,8,No,100,300,Start Time Cannot Vary,16 to 30 Minutes,No
1,3000056,2,Male,35-54 years,Yes,Spouse/Partner,White,Yes,Works,Employed,...,Bergen,Auto Passenger,,90,No,730,1900,Within 15 Minutes or Less,More than 1 Hour,No
2,3000056,3,Male,19-24 years,Yes,Son/Daughter,White,Yes,Works,Employed,...,Fairfield,Auto Passenger,,14,No,1800,2330,More than 1 Hour,More than 1 Hour,Yes - Full Time
3,3000056,4,Male,16-18 years,Yes,Son/Daughter,White,Yes,Works,Employed,...,Fairfield,Auto Passenger,,14,No,1600,2330,More than 1 Hour,More than 1 Hour,Yes - Full Time
4,3000056,5,Female,Younger than 16 years,Not Applicable,Son/Daughter,White,Not Applicable,Not Applicable,Not Applicable,...,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Yes - Full Time


In [32]:
person_clean.to_csv('../../fm-lm-demand-simulation/data/processed/person.csv')

# Merge Datasets

In [33]:
linked_household = pd.merge(linked_clean, household_clean, how='inner', on=['SAMPN'])
trip_survey = pd.merge(linked_household, person_clean, how='inner', on=['SAMPN', 'PERNO'])
trip_survey.head()


,SAMPN,PERNO,PERTYPE,ORIG_HOME,DEST_HOME,DOW_x,OTAZ,DTAZ,DTPURP,OTPURP_AGG,...,WFIPS,WMODE,O_WMODE,TTTWS,CDRIV,WSTRT,WEND,STRVR,ENDVR,STUDE
0,3000056,1,2,1,0,3,2626,2627,24,0,...,Fairfield,Auto Passenger,,8,No,100,300,Start Time Cannot Vary,16 to 30 Minutes,No
1,3000056,1,2,0,1,3,2627,2626,3,9,...,Fairfield,Auto Passenger,,8,No,100,300,Start Time Cannot Vary,16 to 30 Minutes,No
2,3000056,1,2,1,0,3,2626,2627,9,0,...,Fairfield,Auto Passenger,,8,No,100,300,Start Time Cannot Vary,16 to 30 Minutes,No
3,3000056,1,2,0,1,3,2627,2626,3,1,...,Fairfield,Auto Passenger,,8,No,100,300,Start Time Cannot Vary,16 to 30 Minutes,No
4,3000056,1,2,1,0,3,2626,2628,25,0,...,Fairfield,Auto Passenger,,8,No,100,300,Start Time Cannot Vary,16 to 30 Minutes,No


In [34]:
len(trip_survey)

111089

In [36]:
import geopandas as gpd

taz = gpd.read_file('../../fm-lm-demand-simulation/data/external/NYMTC_BPM_TAZ2010_CT2010/BPM_TAZ2010.shp')
taz = taz[['TAZ_ID1', 'geometry']]
taz.head()

,TAZ_ID1,geometry
0,2440,"POLYGON ((-73.95330299999999 41.920894, -73.95..."
1,2438,"(POLYGON ((-73.884192 41.969296, -73.885808 41..."
2,2433,"(POLYGON ((-73.818944 42.046672, -73.818496 42..."
3,2437,"POLYGON ((-73.715968 41.931984, -73.715952 41...."
4,2442,"POLYGON ((-73.77731199999999 41.924592, -73.77..."


In [37]:
trip_survey_shape = pd.merge(trip_survey, taz, how='inner', left_on='OTAZ', right_on='TAZ_ID1')

In [38]:
trip_survey_shape.head()

,SAMPN,PERNO,PERTYPE,ORIG_HOME,DEST_HOME,DOW_x,OTAZ,DTAZ,DTPURP,OTPURP_AGG,...,O_WMODE,TTTWS,CDRIV,WSTRT,WEND,STRVR,ENDVR,STUDE,TAZ_ID1,geometry
0,3000056,1,2,1,0,3,2626,2627,24,0,...,,8,No,100,300,Start Time Cannot Vary,16 to 30 Minutes,No,2626,"POLYGON ((-73.4243 41.4551, -73.42529999999999..."
1,3000056,1,2,1,0,3,2626,2627,9,0,...,,8,No,100,300,Start Time Cannot Vary,16 to 30 Minutes,No,2626,"POLYGON ((-73.4243 41.4551, -73.42529999999999..."
2,3000056,1,2,1,0,3,2626,2628,25,0,...,,8,No,100,300,Start Time Cannot Vary,16 to 30 Minutes,No,2626,"POLYGON ((-73.4243 41.4551, -73.42529999999999..."
3,3000056,2,1,1,0,3,2626,2676,9,0,...,,90,No,730,1900,Within 15 Minutes or Less,More than 1 Hour,No,2626,"POLYGON ((-73.4243 41.4551, -73.42529999999999..."
4,3000056,3,3,1,0,3,2626,2627,27,0,...,,14,No,1800,2330,More than 1 Hour,More than 1 Hour,Yes - Full Time,2626,"POLYGON ((-73.4243 41.4551, -73.42529999999999..."


In [39]:
len(trip_survey_shape)

111089

In [40]:
trip_survey_shape.isnull().sum()

SAMPN              0
PERNO              0
PERTYPE            0
ORIG_HOME          0
DEST_HOME          0
DOW_x              0
OTAZ               0
DTAZ               0
DTPURP             0
OTPURP_AGG         0
DTPURP_AGG         0
LTMODE_AGG         0
TRP_DEP_HR         0
TRP_DEP_MIN        0
TRP_ARR_HR         0
TRP_ARR_MIN        0
TRPDUR_R           0
ACTDUR             0
OTRACT             0
DTRACT             0
PMODE1             0
PMODE_R            0
PMODE_R2           0
PMODE_R3           0
WORK_PURP          0
DTPURP_R           0
ODTPURP            0
ODTPURP1           0
ODTPURP2           0
ODTPURP2_R         0
               ...  
CTFIP              0
INCOM_R            0
DOW_y              0
HHVEH              0
GENDER             0
AGE_R              0
LIC                0
RELAT              0
RACE               0
EMPLY              0
WORKS              0
WKSTAT             0
JOBS               0
WDAYS              0
INDUS              0
OCCUP              0
O_OCCUP      

In [41]:
trip_survey_shape.to_csv('../../fm-lm-demand-simulation/data/processed/trip_survey_working.csv')

In [42]:
trip_survey_shape1 = pd.read_csv('../../fm-lm-demand-simulation/data/processed/trip_survey_working.csv', index_col=0)
trip_survey_shape1.isnull().sum()

/Users/Danny1/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,1,2,3,4,5,6,9,10,11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


SAMPN              0
PERNO              0
PERTYPE            0
ORIG_HOME          0
DEST_HOME          0
DOW_x              2
OTAZ               0
DTAZ               0
DTPURP             0
OTPURP_AGG         0
DTPURP_AGG         0
LTMODE_AGG         0
TRP_DEP_HR         0
TRP_DEP_MIN        4
TRP_ARR_HR         4
TRP_ARR_MIN        4
TRPDUR_R           4
ACTDUR             4
OTRACT             4
DTRACT             4
PMODE1             4
PMODE_R            4
PMODE_R2           4
PMODE_R3           4
WORK_PURP          4
DTPURP_R           4
ODTPURP            4
ODTPURP1           4
ODTPURP2           4
ODTPURP2_R         4
               ...  
CTFIP              4
INCOM_R            4
DOW_y              4
HHVEH              4
GENDER             4
AGE_R              4
LIC                4
RELAT              4
RACE               4
EMPLY              4
WORKS              4
WKSTAT             4
JOBS               4
WDAYS              4
INDUS              4
OCCUP              4
O_OCCUP      

In [43]:
trip_survey_shape1 = trip_survey_shape1[np.isfinite(trip_survey_shape1['OTRACT'])]
trip_survey_shape1 = trip_survey_shape1[np.isfinite(trip_survey_shape1['TAZ_ID1'])]
trip_survey_shape1.isnull().sum()

SAMPN              0
PERNO              0
PERTYPE            0
ORIG_HOME          0
DEST_HOME          0
DOW_x              0
OTAZ               0
DTAZ               0
DTPURP             0
OTPURP_AGG         0
DTPURP_AGG         0
LTMODE_AGG         0
TRP_DEP_HR         0
TRP_DEP_MIN        0
TRP_ARR_HR         0
TRP_ARR_MIN        0
TRPDUR_R           0
ACTDUR             0
OTRACT             0
DTRACT             0
PMODE1             0
PMODE_R            0
PMODE_R2           0
PMODE_R3           0
WORK_PURP          0
DTPURP_R           0
ODTPURP            0
ODTPURP1           0
ODTPURP2           0
ODTPURP2_R         0
               ...  
CTFIP              0
INCOM_R            0
DOW_y              0
HHVEH              0
GENDER             0
AGE_R              0
LIC                0
RELAT              0
RACE               0
EMPLY              0
WORKS              0
WKSTAT             0
JOBS               0
WDAYS              0
INDUS              0
OCCUP              0
O_OCCUP      

In [44]:
len(trip_survey_shape1)

111085

#### Process Features
* Age - <16, 16-55, 55+

* Number of cars – 0, 1, 2+

* Work flexible – 0, 1 (working starting time)

* Parking – PLOC

* Are users In couple - (0,1)

* Children – (0,1)

* TAZ characteristics  - income


In [45]:
# Replace Values
#vals_to_replace = {'16-18 years':'16-55 years', '19-24 years':'16-55 years', '25-34 years':'16-55 years', 
                   #'35-54 years':'16-55 years', '55-64 years':'55+', '65 years or older':'55+', 'Younger than 16 years':'<16'}

#trip_survey_shape1.AGE_R = trip_survey_shape1.AGE_R.map(vals_to_replace)
#trip_survey_shape1.AGE_R.unique()

In [46]:
# Replace Values
vals_to_replace = {0:0, 1:1, 2:'2+', 3:'2+', 4:'2+', 5:'2+', 6:'2+', 7:'2+', 8:'2+'}

trip_survey_shape1.HHVEH = trip_survey_shape1.HHVEH.map(vals_to_replace)
trip_survey_shape1.HHVEH.unique()

array(['2+', 1, 0], dtype=object)

In [47]:
# Create work flexible column
work_flexible_start = []
work_flexible_end = []
time_variations = ['16 to 30 Minutes', 'More than 1 Hour', '31 to 60 Minutes']

for index,row in trip_survey_shape1.iterrows():
    if row['STRVR'] in time_variations:
        work_flexible_start.append(1)
    else:
        work_flexible_start.append(0)
        
for index,row in trip_survey_shape1.iterrows():
    if row['ENDVR'] in time_variations:
        work_flexible_end.append(1)
    else:
        work_flexible_end.append(0)


In [48]:
trip_survey_shape1['work_flex_start'] = work_flexible_start
trip_survey_shape1['work_flex_end'] = work_flexible_end

In [49]:
trip_survey1 = trip_survey_shape1.drop(['TAZ_ID1', 'geometry'], axis=1)

In [50]:
trip_survey_shape1.to_csv('../../fm-lm-demand-simulation/data/processed/trip_survey_working_shape.csv')
trip_survey1.to_csv('../Data/trip_survey.csv')